In [1]:
!pip install sklearn-recommender
!pip install sklearn-evaluation
!pip install black
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import time
import joblib
#import Recommenders as Recommenders
import sklearn_recommender as recommenders
import sklearn_evaluation as Evaluation

You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\deekshitha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_c

In [2]:
class item_similar_collaborative():
    def __init__(self):
        self.train_data=None
        self.user_id=None
        self.song_id=None
    def create(self,train_data,user_id,song_id):
        self.train_data=train_data
        self.user_id=user_id
        self.song_id=song_id
    #this function returns all unique songs listened by a given user
    def get_user_songs(self,user):
        user_data=self.train_data[self.train_data[self.user_id]==user]
        user_listened_songs=list(user_data[self.song_id].unique())
        return user_listened_songs
    #this function returns all unique listeners of given song
    def get_listeners_of_givensong(self,song):
        song_data=self.train_data[self.train_data[self.song_id]==song]
        song_listeners=list(song_data[self.user_id].unique())
        return song_listeners
    #this function results all unique songs in our training_data
    def get_unique_songs(self):
        unique_songs=self.train_data[self.song_id].unique()
        return unique_songs
    def listenersOfUserListendSongs(self,user_listened_songs,unique_songs):
        #for each song in our user history we find users who listen to the same song
        user_listened_songs_listeners=[]
        for song in user_listened_songs:
            user_listened_songs_listeners.append(self.get_listeners_of_givensong(song))
        return user_listened_songs_listeners
    def build_item_To_item_cooccurence_matrix(self,user_listened_songs,user_listened_songs_listeners,unique_songs):
        #initialising matrix
        item_To_item_cooccurence_matrix=np.matrix(np.zeros(shape=(len(user_listened_songs),len(unique_songs))))
        #calculating similarity using Jaccard Similarity J(A,B)=intersection(A,B)/union(A,B)
        for i in range(len(unique_songs)):
            song_data=self.train_data[self.train_data[self.song_id]==unique_songs[i]]
            song_listeners=set(song_data[self.user_id].unique())
            for j in range(0,len(user_listened_songs)):
                song_listeners_usersong=user_listened_songs_listeners[j]
                intersection_users=song_listeners.intersection(song_listeners_usersong)
                if(len(intersection_users)!=0):
                    union_users=song_listeners.union(song_listeners_usersong)
                    item_To_item_cooccurence_matrix[j,i]=len(intersection_users)/len(union_users)
        return item_To_item_cooccurence_matrix
    def generate_recommendations(self,all_unique_songs_in_training_data,user_id,item_to_item_cooccurence,user_listened_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(item_to_item_cooccurence))
        #Calculate a weighted average of the scores in cooccurence matrix
        similarity_scores = item_to_item_cooccurence.sum(axis=0)/float(item_to_item_cooccurence.shape[0])
        similarity_scores = np.array(similarity_scores)[0].tolist()
        #sort based on similarity_scores
        sort_index = sorted(((e,i) for i,e in enumerate(list(similarity_scores))), reverse=True)
        #creating new dataframe 
        columns = ['user_id', 'song_id', 'score', 'rank']
        df = pd.DataFrame(columns=columns)
        rank = 1 
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_unique_songs_in_training_data[sort_index[i][1]] not in user_listened_songs and rank <= 10:
                df.loc[len(df)]=[user_id,all_unique_songs_in_training_data[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1
        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
    
    def recommend(self, user_id):
        #all unique songs listened by user
        user_listened_songs = self.get_user_songs(user_id)    
            
        print("No. of unique songs for the user: %d" % len(user_listened_songs))
        
        all_unique_songs_trainingdata = self.get_unique_songs()
        
        print("no. of unique songs in the training set: %d" % len(all_unique_songs_trainingdata))
        user_listened_songs_listeners=self.listenersOfUserListendSongs(user_listened_songs,all_unique_songs_trainingdata)
         
        #build item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        item_to_item_cooccurence_matrix = self. build_item_To_item_cooccurence_matrix(user_listened_songs,user_listened_songs_listeners,all_unique_songs_trainingdata)
        
        df_recommendations = self.generate_recommendations(all_unique_songs_trainingdata,user_id,item_to_item_cooccurence_matrix,user_listened_songs)
                
        return df_recommendations
    
   
    

In [3]:
#collecting the datasets and merging both million_song and song_data
song_df_1=pd.read_table("million_song.txt",header=None)
song_df_1.columns=['user_id','song_id','listen_count']
song_df_2=pd.read_csv("song_data.csv")
song_df=pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']),on="song_id",how="left")
song_df.head()
song_df = song_df.head(10000)

In [4]:
#splitting the data in to training set and testing set
train_data,test_data=train_test_split(song_df,test_size=0.2,random_state=0)
#train_data.head()
test_data.head()

,user_id,song_id,listen_count,title,release,artist_name,year
9394,97e48f0f188e04dcdb0f8e20a29dacb881d80c9e,SOLCAXX12B0B8079A5,1,Mister Jung Stuffed,Rabbit Habits,Man Man,2008
898,a820d2d4f16bbd53be9e41e0417dfb234bfdfba8,SOHMTUC12A58A799D4,2,Ort,Cunning Stunts,Cows,1992
2398,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,SOEUTXF12A6D4FC6F8,2,Suena tu guitarra,Muriendo de Amor,Fernando Soto,0
5906,806ccae96c8ecb1c198482aff785ccd6bbe17143,SOIBSSJ12A8151E066,1,Let Me Be Myself,3 Doors Down,3 Doors Down,2008
2343,8cbb5066924ec788e3fea9a4aae59586f46f38fa,SOPPROJ12AB0184E18,1,Pursuit Of Happiness (nightmare),Pursuit Of Happiness,Kid Cudi / MGMT / Ratatat,2010


In [5]:
users = song_df['user_id'].unique()
is_model = item_similar_collaborative()
is_model.create(train_data, 'user_id', 'song_id')
users

array(['b80344d063b5ccb3212f76538f3d9e43d87dca9e',
       '85c1f87fea955d09b4bec2e36aee110927aedf9a',
       'bd4c6e843f00bd476847fb75c47b4fb430a06856',
       '8937134734f869debcab8f23d77465b4caaa85df',
       '969cc6fb74e076a68e36a04409cb9d3765757508',
       '4bd88bfb25263a75bbdd467e74018f4ae570e5df',
       'e006b1a48f466bf59feefed32bec6494495a4436',
       '9d6f0ead607ac2a6c2460e4d14fb439a146b7dec',
       '9bb911319fbc04f01755814cb5edb21df3d1a336',
       'b64cdd1a0bd907e5e00b39e345194768e330d652',
       '17aa9f6dbdf753831da8f38c71b66b64373de613',
       'd6589314c0a9bcbca4fee0c93b14bc402363afea',
       '5a905f000fc1ff3df7ca807d57edb608863db05d',
       'c737ec8c1b16ce8e39115f4432c9a7fc21ec47a1',
       '45544491ccfcdc0b0803c34f201a6287ed4e30f8',
       'ed7d4c476013b1c3dd91982b61494bf7436083ba',
       'baf47ed8da24d607e50d8684cde78b923538640f',
       '169f9f4c68b62d1887c7c0ac99d10a79cfca5daf',
       'a820d2d4f16bbd53be9e41e0417dfb234bfdfba8',
       'bd8475385f0aa78830fa6df

In [6]:
users = song_df['user_id'].unique()
is_model = item_similar_collaborative()
is_model.create(train_data, 'user_id', 'song_id')
user_id = users[4]
user_items = is_model.get_user_songs(user_id)
#
print("Training data songs for the user userid: %s:" % user_id)

for user_item in user_items:
    print(user_item)

print("your recommendations are loading pls wait")
print("--------------------------------------------------")

#personalized model
is_model.recommend(user_id)

Training data songs for the user userid: 969cc6fb74e076a68e36a04409cb9d3765757508:
SOUFPNI12A8C142D19
SONQEYS12AF72AABC9
SOOCUXJ12A6D4F6FB7
SOQBMFK12A8C13835B
SOXIIIM12A6D4F66C8
SOPXRQT12A6701EAC9
SOABRAB12A6D4F7AAF
SONZWDK12A6701F62B
SOBZZDU12A6310D8A3
SOLVRLL12A67020D7F
SOAOQFD12A6D4FAAA9
SOBFPJC12A58A7D1AB
SOERVXG12A6D4F5583
SORPHNV12A6701CD63
SOPKPFW12A6D4F84BC
SOXHIDK12A58A7CFB3
SODCNJX12A6D4F93CB
SOFXZAW12A67ADC90E
SOGFUFC12A8C13F1E5
your recommendations are loading pls wait
--------------------------------------------------
No. of unique songs for the user: 19
no. of unique songs in the training set: 4502
Non zero values in cooccurence_matrix :1990


,user_id,song_id,score,rank
0,969cc6fb74e076a68e36a04409cb9d3765757508,SOWSWBS12A6D4F658A,0.04386,1
1,969cc6fb74e076a68e36a04409cb9d3765757508,SOIIQPJ12B0B807AF4,0.04386,2
2,969cc6fb74e076a68e36a04409cb9d3765757508,SOVZNUC12AB017F92A,0.04386,3
3,969cc6fb74e076a68e36a04409cb9d3765757508,SOFWIIA12A8C1448C9,0.04386,4
4,969cc6fb74e076a68e36a04409cb9d3765757508,SOXJZBI12AB0182B7C,0.04386,5
5,969cc6fb74e076a68e36a04409cb9d3765757508,SOUVTWC12AB017F941,0.04386,6
6,969cc6fb74e076a68e36a04409cb9d3765757508,SOZVUUB12A8C13B5EF,0.04386,7
7,969cc6fb74e076a68e36a04409cb9d3765757508,SOBUOEK12A6D4F9908,0.04386,8
8,969cc6fb74e076a68e36a04409cb9d3765757508,SOPOMUT12A8C140BAD,0.04386,9
9,969cc6fb74e076a68e36a04409cb9d3765757508,SOOKZPI12A67AD9549,0.04386,10
